In [1]:
import json
import pandas as pd
import numpy as np
import os
from PIL import Image

# 1. 파일 경로 설정
json_path = '/home/bongo/porter_notebook/research/qwen3/results_32B_absolute/results_vlm_exo_selection.json'

In [2]:

with open(json_path, 'r',encoding='utf-16') as f:
    data = json.load(f)
    
# 3. 데이터 평탄화 (Flattening)
# 목표 구조: [{'pair_key': 'hold$axe', 'file_name': 'abc.jpg', 'beat': -3.5, ...}, ...]
rows = []

for pair_key, files_dict in data.items():
    for file_name, scores_dict in files_dict.items():
        # 기본 메타데이터 생성
        row_data = {
            'pair_key': pair_key,   # 예: 'hold$axe'
            'file_name': file_name  # 예: 'hold_axe_000384.jpg'
        }
        # 액션 점수 딕셔너리를 합침 (Action Score 컬럼들 추가)
        row_data.update(scores_dict)
        
        rows.append(row_data)

# 4. DataFrame 생성
df = pd.DataFrame(rows)
df[['action', 'object_name']] = df['pair_key'].str.split('$', expand=True)
AGD20K_PATH = "/home/DATA/AGD20K"
df['exo_path'] = df.apply(lambda x: f"{AGD20K_PATH}/Seen/trainset/exocentric/{x['action']}/{x['object_name']}/{x['file_name']}",axis=1)
df

,pair_key,file_name,beat,boxing,brush_with,carry,catch,cut,cut_with,drag,...,take_photo,talk_on,text_on,throw,type_on,wash,write,action,object_name,exo_path
0,hold$axe,hold_axe_000384.jpg,-3.515625,4.281250,3.312500,3.375000,1.210938,4.15625,4.15625,3.250000,...,-2.343750,-2.812500,3.265625,2.718750,-1.296875,-1.945312,-3.000000,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...
1,hold$axe,hold_axe_000856.jpg,-1.523438,2.875000,3.437500,0.843750,2.656250,12.62500,12.62500,0.550781,...,-2.156250,-2.937500,2.687500,3.359375,-2.203125,-3.031250,-2.781250,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...
2,hold$axe,hold_axe_001123.jpg,-5.843750,1.515625,1.054688,-2.281250,1.757812,5.28125,5.28125,1.726562,...,-3.109375,-4.062500,2.046875,9.312500,-2.593750,-4.875000,-3.281250,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...
3,hold$axe,hold_axe_003505.jpg,-2.468750,3.312500,2.875000,0.601562,2.703125,10.75000,10.75000,1.296875,...,-1.218750,-2.421875,3.968750,2.968750,-1.148438,-1.742188,-2.046875,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...
4,hold$axe,hold_axe_006372.jpg,-2.218750,2.875000,4.218750,0.003113,2.859375,15.75000,15.75000,-0.008606,...,-2.234375,-2.390625,2.843750,2.203125,-1.937500,-3.359375,-2.375000,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,push$bicycle,push_bicycle_013304.jpg,-0.326172,6.718750,6.000000,5.406250,4.718750,9.87500,9.87500,4.218750,...,0.800781,0.034424,5.312500,2.625000,1.421875,-0.542969,0.453125,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...
91,push$bicycle,push_bicycle_013392.jpg,-2.625000,4.843750,6.312500,10.687500,3.296875,6.53125,6.53125,10.187500,...,-0.511719,0.625000,5.093750,3.265625,-0.219727,0.003510,-2.296875,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...
92,push$bicycle,push_bicycle_013423.jpg,-0.632812,7.125000,6.375000,3.671875,4.625000,8.31250,8.31250,6.093750,...,0.644531,0.601562,5.312500,3.656250,1.562500,-1.304688,-0.707031,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...
93,push$bicycle,push_bicycle_013710.jpg,-2.593750,4.312500,7.000000,11.625000,5.562500,8.18750,8.18750,8.562500,...,-0.337891,-0.519531,4.687500,3.453125,0.527344,0.190430,-1.585938,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...


In [3]:
def get_image_area_from_dir(dir_path):
    """
    폴더 경로를 입력받아 내부의 첫 번째 이미지를 찾고
    (너비, 높이, 넓이)를 반환합니다.
    """
    # 1. 경로 존재 여부 확인
    if not os.path.exists(dir_path):
        return None, None, None
    
    # 2. 이미지 확장자 필터링
    valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.PNG', '.JPG')
    

        
    # 3. 첫 번째 이미지 열기 (파일 전체를 로드하지 않고 헤더만 읽어서 빠름)
    first_img_path = os.path.join(dir_path)
    with Image.open(first_img_path) as img:
        w, h = img.size
        area = w * h
        return w, h, area
            
        
df[['exo_w', 'exo_h', 'exo_area']] = df['exo_path'].apply(
    lambda path: pd.Series(get_image_area_from_dir(path))
)
df

,pair_key,file_name,beat,boxing,brush_with,carry,catch,cut,cut_with,drag,...,throw,type_on,wash,write,action,object_name,exo_path,exo_w,exo_h,exo_area
0,hold$axe,hold_axe_000384.jpg,-3.515625,4.281250,3.312500,3.375000,1.210938,4.15625,4.15625,3.250000,...,2.718750,-1.296875,-1.945312,-3.000000,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,983,1500,1474500
1,hold$axe,hold_axe_000856.jpg,-1.523438,2.875000,3.437500,0.843750,2.656250,12.62500,12.62500,0.550781,...,3.359375,-2.203125,-3.031250,-2.781250,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1500,2250000
2,hold$axe,hold_axe_001123.jpg,-5.843750,1.515625,1.054688,-2.281250,1.757812,5.28125,5.28125,1.726562,...,9.312500,-2.593750,-4.875000,-3.281250,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1172,1758000
3,hold$axe,hold_axe_003505.jpg,-2.468750,3.312500,2.875000,0.601562,2.703125,10.75000,10.75000,1.296875,...,2.968750,-1.148438,-1.742188,-2.046875,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1240,1280,1587200
4,hold$axe,hold_axe_006372.jpg,-2.218750,2.875000,4.218750,0.003113,2.859375,15.75000,15.75000,-0.008606,...,2.203125,-1.937500,-3.359375,-2.375000,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1000,1500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,push$bicycle,push_bicycle_013304.jpg,-0.326172,6.718750,6.000000,5.406250,4.718750,9.87500,9.87500,4.218750,...,2.625000,1.421875,-0.542969,0.453125,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,556,355840
91,push$bicycle,push_bicycle_013392.jpg,-2.625000,4.843750,6.312500,10.687500,3.296875,6.53125,6.53125,10.187500,...,3.265625,-0.219727,0.003510,-2.296875,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,573,366720
92,push$bicycle,push_bicycle_013423.jpg,-0.632812,7.125000,6.375000,3.671875,4.625000,8.31250,8.31250,6.093750,...,3.656250,1.562500,-1.304688,-0.707031,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,430,275200
93,push$bicycle,push_bicycle_013710.jpg,-2.593750,4.312500,7.000000,11.625000,5.562500,8.18750,8.18750,8.562500,...,3.453125,0.527344,0.190430,-1.585938,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,485,310400


In [4]:
# 1. 점수 컬럼 이름들만 리스트로 뽑기
# (메타데이터 컬럼들은 제외합니다)
meta_cols = ['pair_key', 'file_name',  'action', 'object_name', 'exo_path', 'exo_w', 'exo_h','exo_area']
score_cols = [c for c in df.columns if c not in meta_cols]

# 2. 점수 데이터만 따로 꺼내서 Numpy 배열로 변환
raw_scores_matrix = df[score_cols].values.astype(float)

# 3. Softmax 일괄 계산 (행렬 연산)
# axis=1 은 '가로 방향(행)'으로 계산한다는 뜻입니다.
# (Overflow 방지를 위해 각 행의 최댓값을 빼줍니다)
max_vals = np.max(raw_scores_matrix, axis=1, keepdims=True)
exp_matrix = np.exp(raw_scores_matrix - max_vals)
probs_matrix = exp_matrix / np.sum(exp_matrix, axis=1, keepdims=True)

# 4. 결과를 담을 새로운 DataFrame 생성 (df_prob)
# 메타데이터는 그대로 복사해오고, 점수 부분만 확률로 교체합니다.
df_prob = df[meta_cols].copy() # 메타데이터 복사
prob_df_part = pd.DataFrame(probs_matrix, columns=score_cols, index=df.index) # 확률 데이터프레임
df_prob = pd.concat([df_prob, prob_df_part], axis=1) # 합치기
df_prob

,pair_key,file_name,action,object_name,exo_path,exo_w,exo_h,exo_area,beat,boxing,...,stick,stir,swing,take_photo,talk_on,text_on,throw,type_on,wash,write
0,hold$axe,hold_axe_000384.jpg,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,983,1500,1474500,2.797137e-08,6.805398e-05,...,8.751467e-08,1.064543e-03,1.258872e-05,9.029269e-08,5.650372e-08,2.464752e-05,1.426489e-05,2.572199e-07,1.344906e-07,4.684323e-08
1,hold$axe,hold_axe_000856.jpg,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1500,2250000,3.578032e-07,2.909788e-05,...,1.317767e-06,6.029988e-04,7.742658e-04,1.900278e-07,8.700108e-08,2.412299e-05,4.723052e-05,1.813258e-07,7.921538e-08,1.017146e-07
2,hold$axe,hold_axe_001123.jpg,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1172,1758000,7.326955e-10,1.150958e-06,...,1.503393e-09,7.623378e-06,9.969637e-01,1.128359e-08,4.350213e-09,1.957845e-06,2.800266e-03,1.889647e-08,1.930395e-09,9.501739e-09
3,hold$axe,hold_axe_003505.jpg,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1240,1280,1587200,9.001440e-07,2.917944e-04,...,6.664218e-06,4.709323e-03,1.844197e-03,3.141811e-06,9.433428e-07,5.624484e-04,2.069132e-04,3.370671e-06,1.861462e-06,1.372556e-06
4,hold$axe,hold_axe_006372.jpg,hold,axe,/home/DATA/AGD20K/Seen/trainset/exocentric/hol...,1500,1000,1500000,7.856254e-09,1.280569e-06,...,3.452828e-08,1.293329e-05,3.407468e-05,7.734454e-09,6.615629e-09,1.241170e-06,6.540507e-07,1.040785e-08,2.511009e-09,6.719810e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,push$bicycle,push_bicycle_013304.jpg,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,556,355840,2.273874e-10,2.608177e-07,...,2.300677e-10,2.553121e-06,2.822981e-10,7.017717e-10,3.261152e-10,6.391612e-08,4.349547e-09,1.305971e-09,1.830681e-10,4.956906e-10
91,push$bicycle,push_bicycle_013392.jpg,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,573,366720,3.900837e-14,6.835885e-11,...,3.818494e-13,6.348800e-09,1.935119e-13,3.228080e-13,1.006039e-12,8.777450e-11,1.410665e-11,4.322697e-13,5.403872e-13,5.415776e-14
92,push$bicycle,push_bicycle_013423.jpg,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,430,275200,1.768984e-11,4.139036e-08,...,3.569959e-11,4.180278e-07,4.789851e-11,6.345522e-11,6.078638e-11,6.756790e-09,1.289556e-09,1.589045e-10,9.035088e-12,1.642446e-11
93,push$bicycle,push_bicycle_013710.jpg,push,bicycle,/home/DATA/AGD20K/Seen/trainset/exocentric/pus...,640,485,310400,2.489927e-12,2.486181e-09,...,3.864590e-11,6.893492e-07,1.681732e-11,2.376259e-11,1.981565e-11,3.617373e-09,1.052715e-09,5.644953e-11,4.030328e-11,6.821407e-12


In [6]:
# 1. 'hold$axe' 데이터만 필터링
target_df = df_prob[df_prob['pair_key'] == 'hold$badminton_racket'].copy()

# 2. 점수(확률) 컬럼들만 골라내기
# (메타데이터 컬럼 제외)
meta_cols = ['pair_key', 'file_name', 'action', 'object_name', 'gt_score', 'softmax_score', 'gap', 'z_score', 'exo_path', 'exo_w', 'exo_h','exo_area']
score_cols = [c for c in target_df.columns if c not in meta_cols]

# 3. 각 행에서 최대값을 가진 '컬럼 이름' 추출 (가장 높은 확률의 행동)
target_df['predicted_action'] = target_df[score_cols].idxmax(axis=1)

# 4. (선택) 그때의 '확률 값'도 추출
target_df['max_prob'] = target_df[score_cols].max(axis=1)

# 5. 결과 확인
# file_name, 정답(action), 예측(predicted_action), 확률(max_prob) 순으로 출력
result_view = target_df[['file_name', 'action', 'predicted_action', 'max_prob','exo_area']]
result_view['final_score'] =  result_view['max_prob'] * result_view['exo_area']
result_view = result_view[result_view['predicted_action']==result_view['action']]
print("=== Prediction Result for 'hold$axe' ===")
result_view.sort_values('max_prob')

=== Prediction Result for 'hold$axe' ===


/tmp/ipykernel_1821710/4292024629.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_view['final_score'] =  result_view['max_prob'] * result_view['exo_area']


,file_name,action,predicted_action,max_prob,exo_area,final_score
21,hold_badminton_racket_003632.jpg,hold,hold,0.982007,345600,339381.719638
31,hold_badminton_racket_011723.jpg,hold,hold,0.989482,272640,269772.259390
27,hold_badminton_racket_008125.jpg,hold,hold,0.998706,163000,162789.122561


In [ ]:
df_prob[df_prob['pair_key'] == 'hold$axe'][['file_name','hold','hit']]*10000

,file_name,hold,hit
0,hold_axe_000384.jpghold_axe_000384.jpghold_axe...,9985.416864,0.001957
1,hold_axe_000856.jpghold_axe_000856.jpghold_axe...,0.003232,0.041593
2,hold_axe_001123.jpghold_axe_001123.jpghold_axe...,0.025535,0.000348
3,hold_axe_003505.jpghold_axe_003505.jpghold_axe...,0.018908,0.201696
4,hold_axe_006372.jpghold_axe_006372.jpghold_axe...,0.000049,0.002387
5,hold_axe_006509.jpghold_axe_006509.jpghold_axe...,9996.540377,0.000389
6,hold_axe_006933.jpghold_axe_006933.jpghold_axe...,9997.503052,0.001812
7,hold_axe_008102.jpghold_axe_008102.jpghold_axe...,9926.657870,0.013823
8,hold_axe_008636.jpghold_axe_008636.jpghold_axe...,0.002220,0.043302
9,hold_axe_009211.jpghold_axe_009211.jpghold_axe...,9992.190240,0.001783
